In [1]:
import nbdev

In [2]:
#| default_exp store

In [3]:
#|export
from __future__ import annotations
from typing import List, Callable, TypeVar,  Generic, Sequence

### Types

In [4]:
#| export

T = TypeVar("T")

Subscriber = Callable[[T], None]

Unsubscriber = Callable[[], None]

Updater = Callable[[T], T]


class Readable(Generic[T]):
    def subscribe(self, subscriber: Subscriber[T]) -> Unsubscriber:
        ...

class Writable(Readable, Generic[T]):
    def set(self, value: T) -> None:
        ...
    def update(self, updater: Updater[T]) -> None:
        ...

subscriber_queue: List[Subscriber] = [] # callback list

In [5]:
#| export

def writable(value: T) -> Writable[T]:
    """ Create a writable store with a given value that allows both updating and reading by subscription."""

    def set(new_value: T) -> None:
        nonlocal value
        if new_value != value: value = new_value
        for subscriber in subscriber_queue: subscriber(value)

    def update(fn: Updater[T]) -> None: set(fn(value))

    def subscribe(subscriber: Subscriber[T]) -> Unsubscriber:
        subscriber_queue.append(subscriber)
        subscriber(value)

        def unsubscribe() -> None:
            subscriber_queue.remove(subscriber)

        return unsubscribe

    ret = Writable() # type: ignore
    ret.set = set # type: ignore
    ret.update = update # type: ignore
    ret.subscribe = subscribe

    return ret

In [6]:
#| export

def readable(value: T) -> Readable[T]:
    ret = Readable()
    ret.subscribe = writable(value).subscribe
    return ret

### Export

In [7]:
nbdev.nbdev_export()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)